# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
weather_df = pd.read_csv("output_data/cities.csv")
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Ushuaia,75,AR,1627236302,100,-54.8000,-68.3000,33.46,21.85
1,Albany,94,US,1627236427,74,42.6001,-73.9662,81.27,6.62
2,Busselton,44,AU,1627236428,66,-33.6500,115.3333,58.08,19.44
3,Naze,100,JP,1627236428,88,28.3667,129.4833,77.97,18.72
4,Gimli,18,CA,1627236428,33,50.6336,-96.9907,84.43,16.13
...,...,...,...,...,...,...,...,...,...
572,Santa Lucía,20,ES,1627236585,65,27.9117,-15.5407,69.12,29.93
573,Hongjiang,0,CN,1627236585,94,27.1100,109.9956,73.72,0.54
574,Ardakān,0,IR,1627236586,10,32.3100,54.0175,97.18,6.38
575,Stornoway,53,GB,1627236586,82,58.2093,-6.3865,62.92,10.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]

In [16]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
temp_df=weather_df.loc[(weather_df['Max Temp']>=70)&(weather_df['Max Temp']<=80)]
wind_file=temp_df.loc[(temp_df['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
132,Saint-François,0,FR,1627236461,64,46.4154,3.9054,75.18,5.75
177,Marsá Maţrūḩ,0,EG,1627236473,59,31.3525,27.2453,78.60,7.52
186,Grand Gaube,0,MU,1627236476,60,-20.0064,57.6608,70.38,5.99
293,Dundee,0,GB,1627236211,72,56.5000,-2.9667,72.81,6.91
332,Cap Malheureux,0,MU,1627236515,60,-19.9842,57.6142,70.47,5.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "Aap_uEClZeEcSG1aZ5i8fNOr2Uo4daWrectRWPtBrq8FiefXJixi_McEJHaSBpw6zKO1QyLOyp6RqmbSqobTrK5esX8tn4rQOqDKCARY7dlGkJfv-vlgEeNzDayGWbAGC0YlmcY0Omh3yJQ35qniIZgDUNBC1JwfNoDJf_dkY63EOcq_pJzoSVPBU0kGlSx_iV6jBufs8Ud8xOgoXM0a3a8fXaB-ACF14QiiG1MCGGfW6n7nh0P6i2gDz93Z32ANaqzUU78Ir-TyPisOri1vJJBzaw3ClL1VGR9x6pRURYY1jy9f5WzvOizvjpgcQ2P2ChYCmWxlC0x_83wuE2lr-obrSh1FY9WL9GXYqR9ZOsBr0TPBgMlKJ-PMY_RwcEURWYgYgEx_w7Wb2Tx7uPP9gZ6O62At524c372lqHJet_c1S1QWJRaynmlxnDXr-OjvFDqO",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 46.23978200000001,
                    "lng": 3.4889
                },
                "viewport": {
                    "northeast": {
                        "lat": 46.24119882989273,
                        "lng": 3.490228479892723
                    },
                    "southwest": {
                        "lat": 46.238499

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Saint-François,0,FR,1627236461,64,46.4154,3.9054,75.18,5.75,Château de Saint Alyre
1,Marsá Maţrūḩ,0,EG,1627236473,59,31.3525,27.2453,78.60,7.52,Jaz Almaza Beach Resort
2,Grand Gaube,0,MU,1627236476,60,-20.0064,57.6608,70.38,5.99,LUX Grand Gaube Resort & Villas
3,Dundee,0,GB,1627236211,72,56.5000,-2.9667,72.81,6.91,Atholl Palace Hotel
4,Cap Malheureux,0,MU,1627236515,60,-19.9842,57.6142,70.47,5.99,LUX Grand Gaube Resort & Villas
5,Kokkola,0,FI,1627236528,69,63.8385,23.1307,72.03,3.00,Hotel Rantakalla
6,Ilhabela,0,BR,1627236545,51,-23.7781,-45.3581,74.57,4.32,DPNY Beach Hotel & SPA
7,Shingū,0,JP,1627236553,80,33.7333,135.9833,74.91,5.08,ResortHotel&Spa BlueMermaid
8,Georgiyevka,0,KZ,1627236554,38,43.0369,74.7114,78.84,6.71,Supara Chunkurchak
9,Liverpool,0,GB,1627236333,64,53.4106,-2.9779,74.68,5.01,Bliss Hotel Southport


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Display figure

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))